## The Bias-Variance Tradeoff
The bias-variance tradeoff is one of the fundamental concepts in supervised machine learning. 

In this chapter, 
   - how to diagnose the problems of overfitting and underfitting. 
   - the concept of ensembling where the predictions of several models are aggregated to produce predictions that are more robust.
   
#### Instantiate the model
 - diagnose the bias and variance problems of a regression tree. 
 - The regression tree will be used to predict the `mpg` consumption of cars from the auto dataset using all available features.

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNN

from sklearn.ensemble import VotingClassifier

In [5]:
mpg= pd.read_csv('auto.csv')
mpg.head()

mpg  displ   hp  weight  accel  origin  size
0  18.0  250.0   88    3139   14.5      US  15.0
1   9.0  304.0  193    4732   18.5      US  20.0
2  36.1   91.0   60    1800   16.4    Asia  10.0
3  18.5  250.0   98    3525   19.0      US  15.0
4  34.3   97.0   78    2188   15.8  Europe  10.0

In [8]:
mpg=pd.get_dummies(mpg)
mpg.head()

mpg  displ   hp  weight  accel  size  origin_Asia  origin_Europe  \
0  18.0  250.0   88    3139   14.5  15.0            0              0   
1   9.0  304.0  193    4732   18.5  20.0            0              0   
2  36.1   91.0   60    1800   16.4  10.0            1              0   
3  18.5  250.0   98    3525   19.0  15.0            0              0   
4  34.3   97.0   78    2188   15.8  10.0            0              1   

   origin_US  
0          1  
1          1  
2          0  
3          1  
4          0

In [10]:
X=mpg.drop('mpg', axis=1)
y=mpg['mpg']

In [11]:
X.head()

displ   hp  weight  accel  size  origin_Asia  origin_Europe  origin_US
0  250.0   88    3139   14.5  15.0            0              0          1
1  304.0  193    4732   18.5  20.0            0              0          1
2   91.0   60    1800   16.4  10.0            1              0          0
3  250.0   98    3525   19.0  15.0            0              0          1
4   97.0   78    2188   15.8  10.0            0              1          0

In [12]:
y.head()

0    18.0
1     9.0
2    36.1
3    18.5
4    34.3
Name: mpg, dtype: float64

In [18]:
SEED =1
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=1)
dt= DecisionTreeRegressor(max_depth=4, min_samples_leaf=0.26)

### Evaluate the 10-fold CV error
Evaluate the 10-fold CV Root Mean Squared Error (RMSE) achieved by the regression tree dt 


In [27]:
MSE_CV_scores = - cross_val_score(dt, X_train, y_train, cv=10, 
                                 scoring='neg_mean_squared_error', n_jobs=-1)
# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean()) ** 0.5
print('CV RMSE: {:.2f}'.format(RMSE_CV))

CV RMSE: 5.14


In [28]:
dt.fit(X_train, y_train)
y_pred_train = dt.predict(X_train)

# Evaluate the training set RMSE of dt
RMSE_train = (MSE(y_train, y_pred_train)) ** 0.5
print("Train RMSE: {:.2f}".format(RMSE_train))

Train RMSE: 5.15


> training error ~ CV error => model underfitting (high bias)

### Ensemble Learning

In [42]:
indian = pd.read_csv('indian_liver_patient/indian_liver_patient_preprocessed.csv', index_col=0)
indian.head()

Age_std  Total_Bilirubin_std  Direct_Bilirubin_std  \
0  1.247403            -0.420320             -0.495414   
1  1.062306             1.218936              1.423518   
2  1.062306             0.640375              0.926017   
3  0.815511            -0.372106             -0.388807   
4  1.679294             0.093956              0.179766   

   Alkaline_Phosphotase_std  Alamine_Aminotransferase_std  \
0                 -0.428870                     -0.355832   
1                  1.675083                     -0.093573   
2                  0.816243                     -0.115428   
3                 -0.449416                     -0.366760   
4                 -0.395996                     -0.295731   

   Aspartate_Aminotransferase_std  Total_Protiens_std  Albumin_std  \
0                       -0.319111            0.293722     0.203446   
1                       -0.035962            0.939655     0.077462   
2                       -0.146459            0.478274     0.203446   
3                       -0.312205            0.293722     0.329431   
4                       -0.177537            0.755102    -0.930414   

   Albumin_and_Globulin_Ratio_std  Is_male_std  Liver_disease  
0                       -0.147390            0              1  
1                       -0.648461            1              1  
2                       -0.178707            1              1  
3                        0.165780            1              1  
4                       -1.713237            1              1

In [43]:
X = indian.drop('Liver_disease', axis=1)
y = indian['Liver_disease']

In [45]:
SEED=1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [46]:


lr = LogisticRegression(random_state=SEED)
knn = KNN(n_neighbors=27)
dt = DecisionTreeClassifier(min_samples_leaf=0.13, random_state=SEED)

# Define the list classifiers
classifiers = [('Logistic Regression', lr), 
               ('K Nearest Neighbours', knn), 
               ('Classification Tree', dt)]

In [47]:
# Iterate over the pre-defined list of classifiers
for clf_name, clf in classifiers:
    # Fit clf to the training set
    clf.fit(X_train, y_train)
    
    # Predict y_pred
    y_pred = clf.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Evaluate clf's accuracy on the test set
    print('{:s} : {:.3f}'.format(clf_name, accuracy))

Logistic Regression : 0.759
K Nearest Neighbours : 0.701
Classification Tree : 0.730


In [50]:
# Import VotingClassifier from sklearn.ensemble


# Instantiate a VotingClassifier vc
vc = VotingClassifier(estimators=classifiers)     

# Fit vc to the training set
vc.fit(X_train, y_train)   

# Evaluate the test set predictions
y_pred = vc.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Voting Classifier: {:.3f}'.format(accuracy))


Voting Classifier: 0.770


> The voting classifier achieves a test set accuracy of 75.3%. This value is greater than that achieved by LogisticRegression.